In [1]:
# import dependencies
import pandas as pd
import math


In [2]:
# read csv on carbon emission data and convert to pandas dataframe
carbon = pd.read_csv('../resources/data/carbon_by_country.csv')

# select only emissions in gigatons of carbon (GtC) by USA since 1969
carbon = carbon[['year', 'USA']].loc[carbon['year'] >= 1969,:]

# rename 'USA' column to 'emissions(GtC)'
carbon = carbon.rename(columns={'USA': 'emissions(GtC)'})

# reset index
carbon = carbon.reset_index(drop=True)


In [3]:
# create funciton to calculate total accumulated carbon emissions(GtC)
def total(x):
    return carbon.loc[carbon['year'] < x,:]['emissions(GtC)'].sum()

# create column with total accumulated carbon emissions for each year
carbon['accumulated(GtC)'] = carbon['year'].apply(total)


In [4]:
# create funciton to calculate scaled dft size
def scale(x):
    return (x-0)/(70730-0)*(524288-16)+16

# create column with scaled dft size for each year
carbon['dft_size'] = carbon['accumulated(GtC)'].apply(scale)


In [5]:
# create funciton to calculate viable fft size
def fft_size(x):
    return 1 if x == 0 else int(2**math.floor(math.log2(x)))

# create column with viable fft size for each year
carbon['fft_size'] = carbon['dft_size'].apply(fft_size)


In [6]:
# create funciton to calculate frequency resolution
def freq_res(x):
    return 44100/x

# create column with frequency resoulution for each year
carbon['freq_res(Hz)'] = carbon['fft_size'].apply(freq_res)


In [9]:
# create funciton to calculate file names
def file(x):
    
    # all unique fft sizes used
    ffts = carbon['fft_size'].unique()
    
    # iterate through unique fft sizes
    for i in range(0, len(ffts)):
        # if the year's fft size matches the indexed fft size:
        if x == ffts[i]:
            # return the file name with the number as the index plus 1
            return f'jimi{i+1}.mp3'

# create column with file name for each year
carbon['file'] = carbon['fft_size'].apply(file)


In [ ]:
# create function to calculate starting times in milliseconds(ms) for each file
def begin(x):
    
    # number of years represented by previous files
    previous = 0
    
    # iterate through previous files
    for i in range(0, int(x[4])):
        # add the number of years represented by each previous file to previous
        previous = previous + len(carbon.loc[carbon['file'] == f'jimi{i}.mp3',:])

    # return start time as previous years multiplied by scaled length of a year in audio playback(ms)
    return previous*4735

# create column for starting times of each file
carbon['begin(ms)'] = carbon['file'].apply(begin)


In [ ]:
# create function to calculate ending times(ms) for each file
def end(x):
    
    # last file ends at ending of manipulated audio
    if x == 'jimi8.mp3':
        return 241471
    
    # for all other files:
    else:
        # number of years represented by current file
        years = len(carbon.loc[carbon["file"] == x,:])
        
        # final year represented by a file is 10 seconds to allow crossfading into next year
        final_year = 10000
        
        # other years multiplied by scaled length of a year in audio playback(ms)
        other_years = (years-1)*4735
        
        # return end time as begining time plus scaled length of years represented by file
        return begin(x) + final_year + other_years

# create column for ending times of each file
carbon['end(ms)'] = carbon['file'].apply(end)


In [ ]:
# display full dataframe
carbon


In [ ]:

# for x in carbon['file'].unique():
#     years = len(carbon.loc[carbon["file"] == x,:])
    
#     previous = 0
    
#     for i in range(0, int(x[4])):
#         previous = previous + len(carbon.loc[carbon['file'] == f'jimi{i}.mp3',:])
    
#     begin = previous*4735
    
#     end = begin + (years-1)*4735 + 10000
    
#     print(f'jimi{i}.mp3 ({years}): {previous}, {begin}, {end}')
    
#     print(int(x[4]))


In [ ]:
# # def begin(x, y):
# #     return x
    
# # carbon['begin'] = carbon[['fft_size','year']].apply(begin)

# # carbon.loc[carbon['file'] == 'jimi1.mp3', :]
# carbon['file'].value_counts()

# # for x in carbon['file'].unique():
# #     len(carbon.loc[carbon['file'] == x,:])

# for i in range(1, len(carbon['file'].unique())+1):
#     years = len(carbon.loc[carbon["file"] == f"jimi{i}.mp3",:])
    
#     previous = 0
    
    
#     for j in range(0, i):
#         previous = previous + len(carbon.loc[carbon['file'] == f'jimi{j}.mp3',:])
    
#     begin = previous*4735
    
#     end = begin + (years-1)*4735 + 10000
    
#     print(f'jimi{i}.mp3 ({years}): {previous}, {begin}, {end}')
        
    
# #         print(len(carbon.loc[carbon['file'] == f'jimi{i}.mp3',:]))



In [ ]:
# # create lists to hold fft sizes and corresponding file name
# fft_size = []
# file = []

# # variables for base fft size and file name number to be multiplied or added to
# fft = 8
# jimi = 0

# # determine fft size and corresponding file name from scaled dft size
# for s in carbon['dft_scaled']:
    
#     # if scaled dft size is greater than or equal to next viable fft size:
#     if s >= fft:
#         # fft = next viable fft size
#         fft = fft*2
#         # file name number increases by 1
#         jimi = jimi+1
#         # add fft size and file name to respective lists
#         fft_size.append(fft)
#         file.append(f'jimi{jimi}.mp3')
        
#     else:
#         # add unaltered fft size and file name to respective lists
#         fft_size.append(fft)
#         file.append(f'jimi{jimi}.mp3')

# # create columns for fft sizes and file names
# carbon['fft_size'] = fft_size
# carbon['file'] = file

# display dataframe
# carbon


In [ ]:
# # group years by file name and display corresponding fft size
# files_group = carbon.groupby('files')

# jimi_fft = files_group['fft_size'].unique()
# jimi_res = files_group['freq_res'].unique()

# jimis = pd.DataFrame({'fft_size': jimi_fft, 'freq_res': jimi_res})

# jimis['fft_size'] = jimis['fft_size'].apply(lambda x: x[0])
# jimis['freq_res'] = jimis['freq_res'].apply(lambda x: x[0])


# jimis

# # # sort from smallest to largest fft size
# # files = files.sort_values('fft_size')

# # # display dataframe
# # files


In [ ]:
# export dataframe as csv
# carbon.to_csv('../resources/data/carbon.csv')
